## Setting up Lysoszyme to run standard MD smulation
This notebook will guide you through how to use FESetup for setting up a protein simulation and generate output for various different simulation engines. 


The notebook forms part of the CCPBio-Sim workshop **Tackling Alchemistry with FESetup and Sire SOMD** run on the 10th of April 2018 at the University of Bristol.

*Author: Anotnia Mey   
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: xx mins**

### Imports

In [2]:
%pylab inline
import nglview as nv

Populating the interactive namespace from numpy and matplotlib


## Setting up Lysozyme 
FESetup has a nice feature, where it will produce compatible output for different MD engines. It currently support *NAMD*, *AMBER*, *Gromacs* and *DLPOLY*, provided the executables for each of the simulation packages are actually installed. In the following we will look at how to set up lysozyme. The first part of this tutorial is found in the directory Task01. 

In [4]:
cd Task01

/Users/toni_brain/Projects/git/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task01


Let's start by loading the molecule.

In [7]:
view = nv.show_file('proteins/181L/protein.pdb')
view

A Jupyter Widget

As for the ethane methanol simulations we have an input file for FESetup. This time it has a new section named [protein]. This contains all the relevant information for setting up a protein simulation. Let's have a look at this very basic setup file. 

In [8]:
!head -n 30 setup.in

# T4-lysozyme: setup

logfile = T4-lysozyme_molsetup.log
forcefield = amber, ff14SB, tip3p, cm

mdengine = amber, sander

# prepare the biomolecules (could also be DNA for example)
[protein]
basedir = proteins
molecules = 181L 
neutralize = True 

box.type = rectangular
box.length = 12.0
align_axes = True

min.nsteps = 200
min.ncyc = 100



We want to put the protein 181L into a box of length 12 add some solvent and neutralize the box and run a set of minimisation steps. Executing this will generate the ouptut file `_proteins` which will contain the minimised protein structure. 

In [ ]:
!FESetup setup.in

In [ ]:
ls _proteins/181L

This directory now contains your solvated protein ready for an equilibration or other type of simulation run. This is the simplest way of preparing a protein simulation. Of course you can again use 

### Setup with Gromacs
We can use the same setup to run with gromacs. The only thin that will have to be changed is the line specifying the MD engine:   
`
mdengine = gromacs, mdrun
`   
If Gromacs is not installed the setup with fail. This is currently the case on this server, but we can still run an example file until we get the error. This time we use `182L` for the setup. 

In [ ]:
!FEsetup setup_gromacs.in

Let's have a look in the `_proteins/182L` directory. You will find it contains a `solvated.gro` and `solvated.top` file, which are the respective coordinate and topology files associated with Gromacs and can then be used to run your simulation with gromacs.  

**Task: What would you have to change to prepare files compatible with NAMD?**

## Setting up Lysozyme with a ligand 
Setting up a single protein isn't necessarily such a complcated task. But setting up a protein and a ligand together might require more effort. The ligand will often lack parameters so partial charges for the forcefield need to be generated and doing this by hand is cumbersome. FESetup can help with this. Let's look at Task02 working with both protein and ligand. 

### Benzene as a ligand of Lysozyme

In [12]:
cd ../Task02

/Users/toni_brain/Projects/git/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task02


You now have a proteins and ligands directory. The proteins directory contains again the structure of 181L and the ligands directory that of benzene.    

**Task: visualise both the protein and the ligand to make sure that the ligand has the right coordinates**

Now we want to actually greate a complex of the ligand and protein and solvate it. For this there is another `setup.in` file prepared. Let's look at this file a bit more closely:

In [13]:
!head -n 50 setup.in

# T4-lysozyme: 

logfile = T4-lysozyme_molsetup.log
forcefield = amber, ff14SB, tip3p, cm

mdengine = amber, sander

#setus up ligand benzene in a solvated box and in vacuum
[ligand]
basedir = ligands
file.name = ligand.mol2
molecules = benzene

box.type = rectangular
box.length = 12.0
neutralize = yes
min.nsteps = 100

# prepare the biomolecules (could also be DNA for example)
[protein]
basedir = proteins
molecules = 181L 
neutralize = True


# create the complex form the ligands and proteins above, solvate and minimise
# the complexes
[complex]
# explicit enumeration of pairs, otherwise all-with-all creation even
# when there is no [complex] section
pairs = 181L : benzene

box.type = rectangular
box.length = 12.0
align_axes = True

min.nsteps = 200
min.ncyc = 100



We again have a ligand section which defines the moleucle  and also makes sure that benzene will be set up in a solvation box. Then again the same protein section, but now in addition there is the complex section, which defines the fact that we want to create a comple of 181L and benzene, but that in a box solvate it and minimise it. 

In [ ]:
# Insert the code here to run the setup. 


Now you should be able to located a directory called `_complex`, that was generated by FESetup. It should contain the complex in vacuum and in solvent. 

In [ ]:
# Insert code to visualise the complex in vacuum



## Butylbenzene as a ligand of Lysosyme
Often you are in a position where you might want to run a simulation with a ligand where you only have a smiles string or sdf file. So how can we go from a smiles file to a solvated complex. One easy way to do this is using babel. 
Take a look at `ligands/ortho-xylene/ligand.smi`. This file contains the following smiles string: `CC1=C(C)C=CC=C1`. What we need to do it generate a 3D structure.   

The next cell uses babel to convert the input smiles string to a 3D structure of format mol2. 

In [ ]:
!obabel -ismi ligands/ortho-xylene/ligand.smi  -opdb ligands/ortho-xylene/ligand.pdb --gen3D -h  > ligands/ortho-xylene/ligand.pdb

Next there is already a perapred input file for ortho-xylene! Let's just run the setup!

In [ ]:
!FESetup setup-xylene.in

## Running MD of Lysozyme with benzene bound
We can now use the generated complex to run standard MD simulations. This could be done with different simulation software. Here we will be using Sire's SOMD. In the directory `Task03/simulation` an input file has already been perpared. SOMD takes standard Amber input files as input. Have a look at what has been provided. 

In [15]:
cd ../Task03

/Users/toni_brain/Projects/git/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task03


In [ ]:
ls simulation

We will take a closer look at the config file. `solvated.rst7` and `solvated.parm7` are the coordinate file and topology file respectively. The complex was prepared with an additional equilibration unsing and NVT and NPT equilibration which has been skipped here. These files are now ready for a production simulation. 

In [ ]:
!cat sim.cfg

As for the simulation, we will run 50 steps of molecular dynamics. 

In [ ]:
## Plot potential energy



In [ ]:
## Visulise the trajectory




## Setting up ligand with alchemical perturbations

In [ ]:
cd ../Task04

## Advanced topics: explicit mapping and restraints. 

## A little quizz for the end
Please have a quick look at this quizz to see what you have learned from this notebook:
https://goo.gl/forms/hSsPTYxnWvKjWj2B3